This dataset was processed as an exercise for the pycon 2018 class taught by Kevin Markham:
  - Youtube: https://www.youtube.com/watch?v=0hsKLYfyQZc&t=1175s
  - Dataset: https://openpolicing.stanford.edu/data/

In [17]:
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
ri = pd.read_csv('data/stanford_open_policing_dataset/ri_statewide_2020_04_01.csv')
ri

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6,17,30) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,raw_row_number,date,time,zone,subject_race,subject_sex,department_id,type,arrest_made,citation_issued,...,reason_for_stop,vehicle_make,vehicle_model,raw_BasisForStop,raw_OperatorRace,raw_OperatorSex,raw_ResultOfStop,raw_SearchResultOne,raw_SearchResultTwo,raw_SearchResultThree
0,1,2005-11-22,11:15:00,X3,white,male,200,vehicular,False,True,...,Speeding,NaN,NaN,SP,W,M,M,NaN,NaN,NaN
1,2,2005-10-01,12:20:00,X3,white,male,200,vehicular,False,True,...,Speeding,NaN,NaN,SP,W,M,M,NaN,NaN,NaN
2,3,2005-10-01,12:30:00,X3,white,female,200,vehicular,False,True,...,Speeding,NaN,NaN,SP,W,F,M,NaN,NaN,NaN
3,4,2005-10-01,12:50:00,X3,white,male,200,vehicular,False,True,...,Speeding,NaN,NaN,SP,W,M,M,NaN,NaN,NaN
4,5,2005-10-01,13:10:00,X3,white,female,200,vehicular,False,True,...,Speeding,NaN,NaN,SP,W,F,M,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509676,509677,2015-08-04,10:40:00,X3,white,male,500,vehicular,False,True,...,Equipment/Inspection Violation,JEEP,GRAND CHEROKEE,EQ,W,M,M,NaN,NaN,NaN
509677,509678,2015-12-20,11:17:00,K3,white,female,300,vehicular,False,True,...,Speeding,NaN,NaN,SP,W,F,M,NaN,NaN,NaN
509678,509679,2015-11-09,23:35:00,X4,white,male,300,vehicular,False,False,...,Equipment/Inspection Violation,FORD,TAURUS,EQ,W,M,W,NaN,NaN,NaN
509679,509680,2015-10-30,11:09:00,X4,white,female,500,vehicular,False,True,...,Speeding,VOLK,BUG,SP,W,F,M,NaN,NaN,NaN


In [19]:
ri.dtypes

raw_row_number            int64
date                     object
time                     object
zone                     object
subject_race             object
subject_sex              object
department_id            object
type                     object
arrest_made              object
citation_issued          object
warning_issued           object
outcome                  object
contraband_found         object
contraband_drugs         object
contraband_weapons       object
contraband_alcohol       object
contraband_other         object
frisk_performed          object
search_conducted           bool
search_basis             object
reason_for_search        object
reason_for_stop          object
vehicle_make             object
vehicle_model            object
raw_BasisForStop         object
raw_OperatorRace         object
raw_OperatorSex          object
raw_ResultOfStop         object
raw_SearchResultOne      object
raw_SearchResultTwo      object
raw_SearchResultThree    object
dtype: o

In [20]:
ri.isnull().sum()

raw_row_number                0
date                         10
time                         10
zone                         10
subject_race              29073
subject_sex               29097
department_id                10
type                          0
arrest_made               29073
citation_issued           29073
warning_issued            29073
outcome                   35841
contraband_found         491919
contraband_drugs         493693
contraband_weapons       497886
contraband_alcohol       508464
contraband_other         491919
frisk_performed              10
search_conducted              0
search_basis             491919
reason_for_search        491919
reason_for_stop           29073
vehicle_make             191564
vehicle_model            279593
raw_BasisForStop          29073
raw_OperatorRace          29073
raw_OperatorSex           29073
raw_ResultOfStop          29073
raw_SearchResultOne      491919
raw_SearchResultTwo      508862
raw_SearchResultThree    509513
dtype: i

## 1. Dropping columns that have more than 20% null values

Let's see which columns will be dropped.

In [21]:
threshold = 0.8 * ri.shape[0]
num_nulls = ri.isnull().sum()
print(f'type(num_nulls): {type(num_nulls)}')

type(num_nulls): <class 'pandas.core.series.Series'>


In [24]:
num_nulls[num_nulls.values > threshold].index.values

array(['contraband_found', 'contraband_drugs', 'contraband_weapons',
       'contraband_alcohol', 'contraband_other', 'search_basis',
       'reason_for_search', 'raw_SearchResultOne', 'raw_SearchResultTwo',
       'raw_SearchResultThree'], dtype=object)

In [25]:
ri.dropna(axis='columns', thresh=threshold, inplace=True)
ri

,raw_row_number,date,time,zone,subject_race,subject_sex,department_id,type,arrest_made,citation_issued,warning_issued,outcome,frisk_performed,search_conducted,reason_for_stop,raw_BasisForStop,raw_OperatorRace,raw_OperatorSex,raw_ResultOfStop
0,1,2005-11-22,11:15:00,X3,white,male,200,vehicular,False,True,False,citation,False,False,Speeding,SP,W,M,M
1,2,2005-10-01,12:20:00,X3,white,male,200,vehicular,False,True,False,citation,False,False,Speeding,SP,W,M,M
2,3,2005-10-01,12:30:00,X3,white,female,200,vehicular,False,True,False,citation,False,False,Speeding,SP,W,F,M
3,4,2005-10-01,12:50:00,X3,white,male,200,vehicular,False,True,False,citation,False,False,Speeding,SP,W,M,M
4,5,2005-10-01,13:10:00,X3,white,female,200,vehicular,False,True,False,citation,False,False,Speeding,SP,W,F,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509676,509677,2015-08-04,10:40:00,X3,white,male,500,vehicular,False,True,False,citation,False,False,Equipment/Inspection Violation,EQ,W,M,M
509677,509678,2015-12-20,11:17:00,K3,white,female,300,vehicular,False,True,False,citation,False,False,Speeding,SP,W,F,M
509678,509679,2015-11-09,23:35:00,X4,white,male,300,vehicular,False,False,True,warning,False,False,Equipment/Inspection Violation,EQ,W,M,W
509679,509680,2015-10-30,11:09:00,X4,white,female,500,vehicular,False,True,False,citation,False,False,Speeding,SP,W,F,M


## 2. Do men or women speed more often?

To do this, we can:
  - filter using reason_for_stop == Speeding and citation_issued == True
  - do a groupby of the subject_sex column
  - sum up the number of each type of driver (male/female)

In [57]:
ri.groupby('subject_sex').reason_for_stop.value_counts(normalize=True)

subject_sex  reason_for_stop                 
female       Speeding                            0.657308
             Other Traffic Violation             0.136581
             Equipment/Inspection Violation      0.107055
             Registration Violation              0.043077
             Seatbelt Violation                  0.027071
             Call for Service                    0.018057
             Special Detail/Directed Patrol      0.005071
             Motorist Assist/Courtesy            0.002532
             Violation of City/Town Ordinance    0.001647
             APB                                 0.000831
             Suspicious Person                   0.000564
             Warrant                             0.000206
male         Speeding                            0.522364
             Other Traffic Violation             0.206948
             Equipment/Inspection Violation      0.135102
             Registration Violation              0.040581
             Special Detai

Problem here is we don't know how many miles women drove vs men. We cannot conclude if men or women are safer drivers from this data alone.

## 3. Does gender affect who gets searched during a stop?

In [58]:
ri.groupby('subject_sex').search_conducted.value_counts(normalize=True)

subject_sex  search_conducted
female       False               0.981249
             True                0.018751
male         False               0.956208
             True                0.043792
Name: search_conducted, dtype: float64

We can also do this using the following code. This is because, the mean is an average of the True=1 and False=0, so it gives you the same number for boolean columns as above. This is bad in the sense that it is not explicit, so a reader of your code may not understand what is going on - avoid it.

In [59]:
ri.groupby('subject_sex').search_conducted.mean()

subject_sex
female    0.018751
male      0.043792
Name: search_conducted, dtype: float64

All we've looked at is gender. We don't know why someone was stopped. If you interview police officers, maybe they will tell you that registration violations require a search. And women may have less registration violations than men. So registration violations is a confounding variable. Let's see if this is related to the type of violation.

ri.groupby(['reason_for_stop', 'subject_sex']) \
    .search_conducted.mean() \
    .unstack()                   # Unstack allows us to move the male/female index 
                                 # into columns. This makes it easier to read the table.

You can see that men are searched more often than women for all reasons for stops. This is just a data point, but does not say that because they're male, they're searched. It is tough to say any relationship is causal, so we will not go there. 

## 4. Why is search_type missing so often?

Why is reason_for_search missing so many times? 
Maybe because search was not conducted that many times. Let's check.

In [69]:
ri.search_conducted.value_counts()

False    491919
True      17762
Name: search_conducted, dtype: int64

This is the same as the number of nulls in the reason_for_search column.

## 5. During a search, how often is the driver frisked?

In [72]:
ri[ri.search_conducted == True].frisk_performed.value_counts(normalize=True)

True     0.524828
False    0.475172
Name: frisk_performed, dtype: float64

You can also do this using the mean() function

In [74]:
ri[ri.search_conducted == True].frisk_performed.mean()

0.5248282851030289

In [79]:
'During a search, the driver is frisked {:4.2f}% of the time' \
    .format(ri[ri.search_conducted == True].frisk_performed.mean() * 100.)

'During a search, the driver is frisked 52.48% of the time'

## Which year had the least number of stops?

In [102]:
ri_non_null_dates = ri.dropna(axis=0, how='any', subset=['date']).copy()
years = ri_non_null_dates \
            .date\
            .str \
            .extract(r'(\d*)\-\d*\-\d*') \
            .astype('int')
ri_non_null_dates.insert(loc=1, column='year', value=years)
ri_non_null_dates.head()

,raw_row_number,year,date,time,zone,subject_race,subject_sex,department_id,type,arrest_made,citation_issued,warning_issued,outcome,frisk_performed,search_conducted,reason_for_stop,raw_BasisForStop,raw_OperatorRace,raw_OperatorSex,raw_ResultOfStop
0,1,2005,2005-11-22,11:15:00,X3,white,male,200,vehicular,False,True,False,citation,False,False,Speeding,SP,W,M,M
1,2,2005,2005-10-01,12:20:00,X3,white,male,200,vehicular,False,True,False,citation,False,False,Speeding,SP,W,M,M
2,3,2005,2005-10-01,12:30:00,X3,white,female,200,vehicular,False,True,False,citation,False,False,Speeding,SP,W,F,M
3,4,2005,2005-10-01,12:50:00,X3,white,male,200,vehicular,False,True,False,citation,False,False,Speeding,SP,W,M,M
4,5,2005,2005-10-01,13:10:00,X3,white,female,200,vehicular,False,True,False,citation,False,False,Speeding,SP,W,F,M


In [107]:
ri_non_null_dates.groupby('year').year.count().sort_values(ascending=True)

year
2005    14079
2010    42391
2009    43453
2013    44686
2011    45794
2015    47065
2008    49019
2014    50810
2007    53203
2006    58343
2012    60828
Name: year, dtype: int64